In [7]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

WorkspaceException: WorkspaceException:
	Message: No workspaces found with name=udacity-project in all the subscriptions that you have access to.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "No workspaces found with name=udacity-project in all the subscriptions that you have access to."
    }
}

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

experiment_folder="https://chidi-compute.southcentralus.instances.azureml.ms/notebooks/Users/odl_user_131609/train.py"

# Specify parameter sampler
ps = 'BayesianOptimization'

# Specify a Policy
policy = 'TruncationSelection'

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = Estimator(source_directory=experiment_folder,
                      entry_script='train.py',
                      compute_target='local',
                      use_docker=False,
                      script_params = {'--reg_rate': 0.07},
                      conda_packages=['scikit-learn']
                      )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy)

NameError: name 'Estimator' is not defined

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
HyperDriveRun(exp, "chidi-2020", hyperdrive_config)

In [ ]:
import joblib
# Get your best run and save the model from that run.

svm_model_linear = LogisticRegression()
filename = "newScript.sav"
### YOUR CODE HERE ###
joblib.dump(svm_model_linear, filename)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_path ='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = Dataset.Tabular.from_delimited_files(path=web_path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="regression",
    primary_metric="spearman_correlation",
    training_data=ds,
    label_column_name="marital",
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
experiment = Experiment(ws, "training-experiment")
local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run = local_run.get_output()